# Initial AI Agent PPO w CNN (working) Supra Drag Cont Act

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
%cd "J:\git\TensorFlowPSX\Py"

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 2000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'discreteAccel' : False,
  'accelAndBrake' : False,
  'discSteer' : False,
  'contAccelOnly' : True,
  'discAccelOnly' : False,
  'modelMode': 1,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 4, # 1 is High Speed Ring x MR2, 2 is 0-400m x MR2, 3 is 0-400 Supra disc, 4 is 0-400 Supra cont, 
}

j:\git\TensorFlowPSX\Py


In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-05 22:03:51,759	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8266


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig

algo = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=True,
        )
    .training(
        train_batch_size=128,
    )
    .build()
)

#PPOConfig.framework("")

2023-10-05 22:03:54,131	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-10-05 22:03:54,707	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-10-05 22:03:54,708	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `c

(RolloutWorker pid=14668) GT Real Time instantiated
(RolloutWorker pid=14668) GT AI Server instantiated for rtgym
(RolloutWorker pid=14668) starting up on localhost port 9999
(RolloutWorker pid=14668) Waiting for a connection


(RolloutWorker pid=14668) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float64
(RolloutWorker pid=14668)   logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(RolloutWorker pid=14668) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (42, 42)
(RolloutWorker pid=14668)   logger.warn(
2023-10-05 22:04:05,319	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the

(RolloutWorker pid=14668) Connection from ('127.0.0.1', 53550)
(RolloutWorker pid=14668) 6


2023-10-05 22:04:05,473	INFO trainable.py:172 -- Trainable.setup took 10.734 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [6]:
N = 1000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


2023-10-05 22:04:05,631	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=14668, ip=127.0.0.1, actor_id=b2c774546d01669b9774db2e01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001B0001F07C0>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_t

(RolloutWorker pid=14668) reset triggered
(RolloutWorker pid=14668) reload save for track : 5
(RolloutWorker pid=14668) 0 0.0 False


(RolloutWorker pid=14668) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: int64
(RolloutWorker pid=14668)   logger.warn(
(RolloutWorker pid=14668) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
(RolloutWorker pid=14668)   logger.warn(f"{pre} is not within the observation space.")


RayTaskError(ValueError): [36mray::RolloutWorker.apply()[39m (pid=14668, ip=127.0.0.1, actor_id=b2c774546d01669b9774db2e01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001B0001F07C0>)
  File "python\ray\_raylet.pyx", line 1424, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 1364, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\_private\function_manager.py", line 726, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\util\tracing\tracing_helper.py", line 464, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\ray\rllib\utils\actor_manager.py", li